In [31]:
import numpy as np
import pandas as pd
import re
import json
from konlpy.tag import Okt
import tensorflow
from tensorflow.python.keras.preprocessing.sequence import pad_sequences
from tensorflow.python.keras.preprocessing.text import Tokenizer

In [2]:
DATA_IN_PATH = '../data/' #데이터가 존재하는 파일 경로
#
train_data = pd.read_csv(DATA_IN_PATH+'ratings_train.txt', header=0, delimiter='\t', quoting=3)

In [3]:
train_data['document'][:5]

0                                  아 더빙.. 진짜 짜증나네요 목소리
1                    흠...포스터보고 초딩영화줄....오버연기조차 가볍지 않구나
2                                    너무재밓었다그래서보는것을추천한다
3                        교도소 이야기구먼 ..솔직히 재미는 없다..평점 조정
4    사이몬페그의 익살스런 연기가 돋보였던 영화!스파이더맨에서 늙어보이기만 했던 커스틴 ...
Name: document, dtype: object

In [6]:
review_text=re.sub("[^가-힣 ㄱ-ㅎ ㅏ-ㅣ \\s]", "", train_data['document'][0]) #한글 아닌 것들 다 제거
print(review_text)
okt = Okt()
review_text = okt.morphs(review_text, stem=True)
print(review_text)

아 더빙 진짜 짜증나네요 목소리
['아', '더빙', '진짜', '짜증나다', '목소리']


In [11]:
#불용어 사전을 재작 후 불용어 제거
stop_words = set(['은', '는', '이', '가', '하', '아', '것', '들', '의', '있', '되', '수', '보', '주', '등', '한'])
clean_review = [token for token in review_text if not token in stop_words]
clean_review

['더빙', '진짜', '짜증나다', '목소리']

In [23]:
#데이터 정제/ 전처리 함수
def preprocessing(review, okt, remove_stopwords=False, stop_words=[]):
    #remove_stopwords: 불용어 제거 여부 선택
    review_text=re.sub("[^가-힣 ㄱ-ㅎ ㅏ-ㅣ \\s]", "", review) #한글 아닌 것들 다 제거
    #review_text ex)아 더빙 진짜 짜증나네요 목소리
    word_review = okt.morphs(review_text, stem=True) #형태소 단위로 분리
    print(word_review) #ex)['아', '더빙', '진짜', '짜증나다', '목소리']
    if remove_stopwords:
        preprocessed_word_review = [token for token in word_review if not token in stop_words]
        #preprocessed_word_rewiew ex)['더빙', '진짜', '짜증나다', '목소리']
    return preprocessed_word_review

In [26]:
#preprocessing함수 test
stop_words = set(['은', '는', '이', '가', '하', '아', '것', '들', '의', '있', '되', '수', '보', '주', '등', '한'])
okt = Okt()
clean_train_review = []

for review in train_data['document'][:4]: #너무 많아서 개수 제한
    if type(review) == str:
        clean_train_review.append(preprocessing(review, okt, remove_stopwords=True, stop_words=stop_words))
    else:
        clean_train_review.append([])
        
clean_train_review[:4]

['아', '더빙', '진짜', '짜증나다', '목소리']
['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다']
['너', '무재', '밓었', '다그', '래서', '보다', '추천', '한', '다']
['교도소', '이야기', '구먼', '솔직하다', '재미', '는', '없다', '평점', '조정']


[['더빙', '진짜', '짜증나다', '목소리'],
 ['흠', '포스터', '보고', '초딩', '영화', '줄', '오버', '연기', '조차', '가볍다', '않다'],
 ['너', '무재', '밓었', '다그', '래서', '보다', '추천', '다'],
 ['교도소', '이야기', '구먼', '솔직하다', '재미', '없다', '평점', '조정']]

In [27]:
#test 데이터도 해준다.
test_data = pd.read_csv(DATA_IN_PATH+'ratings_test.txt', header=0, delimiter='\t', quoting=3)
clean_test_review = []
for review in test_data['document'][:4]: #너무 많아서 개수 제한
    if type(review) == str:
        clean_test_review.append(preprocessing(review, okt, remove_stopwords=True, stop_words=stop_words))
    else:
        clean_test_review.append([])
        
clean_test_review[:4]

['굳다', 'ㅋ']
[]
['뭐', '야', '이', '평점', '들', '은', '나쁘다', '않다', '점', '짜다', '리', '는', '더', '더욱', '아니다']
['지루하다', '않다', '완전', '막장', '임', '돈', '주다', '보기', '에는']


[['굳다', 'ㅋ'],
 [],
 ['뭐', '야', '평점', '나쁘다', '않다', '점', '짜다', '리', '더', '더욱', '아니다'],
 ['지루하다', '않다', '완전', '막장', '임', '돈', '주다', '보기', '에는']]

In [35]:
tokenizer = Tokenizer()
tokenizer.fit_on_texts(clean_train_review)
train_sequences = tokenizer.texts_to_sequences(clean_train_review)
test_sequences = tokenizer.texts_to_sequences(clean_test_review)

word_vocab = tokenizer.word_index
word_vocab["<PAD>"] = 0

MAX_SEQUENCE_LENGTH = 8

train_inputs = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post') #벡터화
train_labels = np.array(train_data['label'])

test_inputs = pad_sequences(test_sequences, maxlen=MAX_SEQUENCE_LENGTH, padding='post') #벡터화
test_labels = np.array(test_data['label'])

In [39]:
DATA_IN_PATH = '../data/' 
TRAIN_INPUT_DATA = 'nsmc_train_input.npy'
TRAIN_LABEL_DATA = 'nsmc_train_label.npy'
TEST_INPUT_DATA = 'nsmc_test_input.npy'
TEST_LABEL_DATA = 'nsmc_test_label.npy'
DATA_CONFIGS = 'data_configs.json'

data_configs = {}

data_configs['vocab'] = word_vocab
data_configs['vocab_size'] = len(word_vocab)+1

import os

if not os.path.exists(DATA_IN_PATH):
    os.makedirs(DEFAULT_PATH + DATA_IN_PATH)
    
np.save(open(DATA_IN_PATH + TRAIN_INPUT_DATA, 'wb'), train_inputs)
np.save(open(DATA_IN_PATH + TRAIN_LABEL_DATA, 'wb'), train_labels)
np.save(open(DATA_IN_PATH + TEST_INPUT_DATA, 'wb'), test_inputs)
np.save(open(DATA_IN_PATH + TEST_LABEL_DATA, 'wb'), test_labels)

json.dump(data_configs, open(DATA_IN_PATH + DATA_CONFIGS, 'w'), ensure_ascii=False)